<a href="https://colab.research.google.com/github/BenFeng666/Testing_1/blob/main/testing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install compatible stack for torch 2.8.0
!pip install -q bitsandbytes==0.42.0
!pip install -q triton==3.4.0
!pip install -q accelerate transformers einops
!pip install -q torchvision


In [ ]:
!pip install -U transformers

In [ ]:
!pip uninstall -y torch torchvision torchaudio bitsandbytes
!pip install torch==2.1.2+cu118 torchvision==0.16.2+cu118 torchaudio==2.1.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install bitsandbytes==0.43.1



Found existing installation: torch 2.8.0
Uninstalling torch-2.8.0:
  Successfully uninstalled torch-2.8.0
Found existing installation: torchvision 0.23.0
Uninstalling torchvision-0.23.0:
  Successfully uninstalled torchvision-0.23.0
Found existing installation: bitsandbytes 0.42.0
Uninstalling bitsandbytes-0.42.0:
  Successfully uninstalled bitsandbytes-0.42.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==2.1.2+cu118 (from versions: 2.2.0, 2.2.0+cpu, 2.2.0+cpu.cxx11.abi, 2.2.0+cu118, 2.2.0+cu121, 2.2.0+rocm5.6, 2.2.0+rocm5.7, 2.2.1, 2.2.1+cpu, 2.2.1+cpu.cxx11.abi, 2.2.1+cu118, 2.2.1+cu121, 2.2.1+rocm5.6, 2.2.1+rocm5.7, 2.2.2, 2.2.2+cpu, 2.2.2+cpu.cxx11.abi, 2.2.2+cu118, 2.2.2+cu121, 2.2.2+rocm5.6, 2.2.2+rocm5.7, 2.3.0, 2.3.0+cpu, 2.3.0+cpu.cxx11.abi, 2.3.0+cu118, 2.3.0+cu121, 2.3.0+rocm5.7, 2.3.0+rocm6.0, 2.3.1, 2.3.1+cpu, 2.3.1+cpu.cxx11.abi, 2.3.1+cu118, 2.3.1+cu121, 2.3.1+rocm5.7, 2.3.1+rocm

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

model_name_or_id = "AI4Chem/ChemLLM-7B-Chat"

# Must be True, otherwise the model won't load
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id, trust_remote_code=True)

prompt = "What is Molecule of Ibuprofen?"
enc = tokenizer(prompt, return_tensors="pt").to("cuda")

# Custom lightweight generate loop (avoids buggy past_key_values)
def simple_generate(model, tokenizer, enc, max_new_tokens=200):
    model.eval()
    input_ids = enc["input_ids"].clone()
    attention_mask = enc["attention_mask"].clone()

    for _ in range(max_new_tokens):
        with torch.inference_mode():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits[:, -1, :]
            next_token = torch.argmax(logits, dim=-1).unsqueeze(-1)

        input_ids = torch.cat([input_ids, next_token], dim=-1)
        attention_mask = torch.cat(
            [attention_mask, torch.ones_like(next_token)], dim=-1
        )

        if next_token.item() == tokenizer.eos_token_id:
            break

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

# Run it
output_text = simple_generate(model, tokenizer, enc, max_new_tokens=200)
print(output_text)







/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

What is Molecule of Ibuprofen? – Molecule of Ibuprofen is the chemical compound of C13H18O2. It is a non-steroidal anti-inflammatory drug (NSAID) used to treat pain, fever, and inflammation. It is also used to treat menstrual cramps, migraines, and rheumatoid arthritis. Ibuprofen is available as over-the-counter (OTC) medication, as well as by prescription. It is available in various forms, including tablets, capsules, liquid, and suppositories. Ibuprofen is a member of the class of drugs known as non-steroidal anti-inflammatory drugs (NSAIDs). NSAIDs are used to treat pain, inflammation, and fever. They work by blocking the production of prostaglandins, which are chemicals that cause pain, inflammation, and fever. Ibuprofen is one of the most widely used NSAIDs. It is available as over-the-counter (OT


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import files
import json

# Upload the file
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Read the Excel file, using the 2nd row as header
df = pd.read_excel(filename, header=1)

# Clean up column names (remove stray spaces)
df.columns = df.columns.str.strip()

# Keep only needed columns
df = df[["Structure", "transfection efficiency (MC3)", "Score"]]
df = df.rename(columns={
    "Structure": "smiles",
    "transfection efficiency (MC3)": "rate",
    "Score": "score"
})

# Drop rows with missing values
df = df.dropna().reset_index(drop=True)

# === Only take the first 210 labeled rows ===
df = df.iloc[:210].copy()

# Train-test split: 150 train, 50 test
train_df, test_df = train_test_split(df, test_size=50, random_state=42, shuffle=True)

# Reset indices
train_df = train_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

print("Train size:", len(train_df), " Test size:", len(test_df))

# Convert to JSON (keep score as float)
train_json = [
    {
        "id": str(i+1),
        "smiles": r.smiles,
        "transfection_efficiency": float(r.rate),
        "score": float(r.score)
    }
    for i, r in train_df.iterrows()
]

test_json = [
    {
        "id": str(i+1),
        "smiles": r.smiles,
        "transfection_efficiency": float(r.rate),
        "score": float(r.score)
    }
    for i, r in test_df.iterrows()
]

# Save to files
with open("train.json", "w") as f:
    json.dump(train_json, f, indent=2)

with open("test.json", "w") as f:
    json.dump(test_json, f, indent=2)

print("Sample train:", train_json[:2])
print("Sample test:", test_json[:2])





Saving Smiles file-Published data.xlsx to Smiles file-Published data (2).xlsx
Train size: 160  Test size: 50
Sample train: [{'id': '1', 'smiles': '[H][C@@]1(OC[C@@H]2CCCCN(CCCCCCCCCCCCCC)CCCCCCCCCCCCCC)[C@@]2([H])OC[C@H]1OCCCN(CCCCCCCCCCCCCC)CCCCCCCCCCCCCC', 'transfection_efficiency': 0.355288614585025, 'score': 1.0}, {'id': '2', 'smiles': 'O=C1C(C(OCCCC)=O)CN(CCCN2CCCC2)CC1C(OCCCC)=O', 'transfection_efficiency': 3.034227, 'score': 1.0}]
Sample test: [{'id': '1', 'smiles': 'OC1=CC=CC=C1CN(CCCN(C)C)CC2=C(O)C=CC=C2', 'transfection_efficiency': 2.900367, 'score': 1.0}, {'id': '2', 'smiles': 'O=C1C(C(OCCCC)=O)CN(CC)CC1C(OCCCC)=O', 'transfection_efficiency': 1.20412, 'score': 1.0}]


In [ ]:
SCORE_SCALE = """
Score reference table:
0 – Slight mRNA transfection efficiency from crude lipid library (0–0.1 fold vs commercial lipid, in vitro data).
1 – Slight mRNA transfection efficiency from purified lipid library (0–0.1 fold vs commercial lipid, in vitro data).
2 – Obvious mRNA transfection efficiency from crude lipid library, less than commercial lipid (0.1–1 fold, in vitro).
3 – Obvious mRNA transfection efficiency from purified lipid library, less than commercial lipid (0.1–1 fold, in vitro).
4 – Obvious mRNA transfection efficiency from crude lipid library, better than commercial lipid (>1 fold), not final candidates (no purified lipid data).
5 – Obvious mRNA transfection efficiency from purified lipid library, better than commercial lipid (>1 fold, in vitro).
6 – Top lipid for in vivo study, not better than commercial lipids (MC3, ALC-0315, SM102).
7 – Top lipid for in vivo study, better than MC3 and ALC-0315.
8 – Top lipid for in vivo study, better than SM102.
"""


In [ ]:
import torch, json, re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ==== 1. Define system prompt with examples + score scale ====
K = 100
few_examples = train_json[:K]

SCORE_SCALE = """
Score reference table:
0 – Slight mRNA transfection efficiency from crude lipid library (0–0.1 fold vs commercial lipid, in vitro data).
1 – Slight mRNA transfection efficiency from purified lipid library (0–0.1 fold vs commercial lipid, in vitro data).
2 – Obvious efficiency from crude lipid library, less than commercial lipid (0.1–1 fold, in vitro).
3 – Obvious efficiency from purified lipid library, less than commercial lipid (0.1–1 fold, in vitro).
4 – Obvious efficiency from crude lipid library, better than commercial lipid (>1 fold), not final candidates (no purified lipid data).
5 – Obvious efficiency from purified lipid library, better than commercial lipid (>1 fold, in vitro).
6 – Top lipid for in vivo study, not better than commercial lipids (MC3, ALC-0315, SM102).
7 – Top lipid for in vivo study, better than MC3 and ALC-0315.
8 – Top lipid for in vivo study, better than SM102.
"""

SYSTEM_PROMPT = (
    "You are a medicinal chemistry and drug delivery assistant.\n\n"
    "Context:\n"
    "I will provide a JSON array of training examples from an assay.\n"
    "Each item is {\"id\": \"...\", \"smiles\": \"...\", "
    "\"transfection_efficiency\": <float>, \"score\": <int>}.\n\n"
    f"Training examples:\n{json.dumps(few_examples, indent=2)}\n\n"
    f"Score scale:\n{SCORE_SCALE}\n\n"
    "Task:\n"
    "Given the following target SMILES, predict:\n"
    "1. A numeric transfection_efficiency (float).\n"
    "2. A score (integer 0–8) based on the scale.\n"
    "3. A confidence value (1–10) for how certain you are about the predicted score.\n"
    "4. A brief rationale (2–5 sentences).\n\n"
    "IMPORTANT OUTPUT RULES:\n"
    "– Line 1: ONLY the float number.\n"
    "– Line 2: ONLY the score (integer).\n"
    "– Line 3: ONLY the confidence (1–10).\n"
    "– Lines 4–8: rationale sentences.\n"
    "– Do NOT write JSON or code.\n"
)

# ==== 2. Helper: build model inputs ====
def build_inputs(smiles: str):
    try:
        prompt_text = tokenizer.apply_chat_template(
            [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"SMILES: {smiles}"}
            ],
            add_generation_prompt=True,
            tokenize=False
        )
    except (ValueError, AttributeError):
        prompt_text = SYSTEM_PROMPT + f"\n\nSMILES: {smiles}\n"

    enc = tokenizer(prompt_text, return_tensors="pt")
    enc = {k: v.to("cuda") for k, v in enc.items()}

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    if getattr(model.generation_config, "pad_token_id", None) is None:
        model.generation_config.pad_token_id = tokenizer.eos_token_id
    return enc

# ==== 3. Run test predictions ====
for i, sample in enumerate(test_json[:20], start=1):
    query_smiles = sample["smiles"]
    true_eff = sample["transfection_efficiency"]
    true_score = sample["score"]

    enc = build_inputs(query_smiles)

    model.eval()
    with torch.inference_mode():
        out = model.generate(
            **enc,
            max_new_tokens=200,
            do_sample=False,
            use_cache=False
        )

    input_len = enc["input_ids"].shape[1]
    gen_text = tokenizer.decode(out[0, input_len:], skip_special_tokens=True).strip()
    lines = [l.strip() for l in gen_text.splitlines() if l.strip()]

    pred_eff, pred_score, pred_conf, rationale = None, None, None, ""

    if len(lines) >= 1:
        m = re.search(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", lines[0])
        if m:
            try: pred_eff = float(m.group(0))
            except: pass
    if len(lines) >= 2:
        m = re.search(r"\d+", lines[1])
        if m:
            try: pred_score = int(m.group(0))
            except: pass
    if len(lines) >= 3:
        m = re.search(r"\d+", lines[2])
        if m:
            try: pred_conf = int(m.group(0))
            except: pass
    if len(lines) > 3:
        rationale = "\n".join(lines[3:]).strip()

    # === Print result ===
    print(f"\n=== Sample {i} ===")
    print("SMILES:", query_smiles)
    print(f"Predicted efficiency: {pred_eff}")
    print(f"Actual efficiency:    {true_eff}")
    print(f"Predicted score:      {pred_score}")
    print(f"Actual score:         {true_score}")
    print(f"Confidence (1–10):    {pred_conf}")
    print("Rationale:", rationale)



=== Sample 1 ===
SMILES: OC1=CC=CC=C1CN(CCCN(C)C)CC2=C(O)C=CC=C2
Predicted efficiency: 0.0
Actual efficiency:    2.900367
Predicted score:      0
Actual score:         1.0
Confidence (1–10):    1
Rationale: Rationale: The SMILES provided is a lipid-like molecule. However, the SMILES does not contain any information about the lipid structure, so it is not possible to predict the transfection efficiency. Therefore, the score is 0, the confidence is 1, and the rationale is "The SMILES provided is a lipid-like molecule. However, the SMILES does not contain any information about the lipid structure, so it is not possible to predict the transfection efficiency. Therefore, the score is 0, the confidence is 1, and the rationale is 'The SMILES provided is a lipid-like molecule. However, the SMILES does not contain any information about the lipid structure, so it is not possible to predict the transfection efficiency. Therefore, the score is 0, the confidence is 1, and the rationale is '".

===

In [ ]:
import json, torch

# Use 150 training examples
training_text = json.dumps(train_json, indent=2)

SYSTEM_PROMPT_TRAIN = """
You are a medicinal chemistry and drug delivery assistant.

Context:
You will first receive 150 training data examples.
Each training example has SMILES, transfection_efficiency, and score.

When you finish reading the 150 training data, ONLY output: "yes".
"""

prompt_text = SYSTEM_PROMPT_TRAIN + f"\n\nHere are the 150 training examples:\n\n{training_text}\n\n"

enc = tokenizer(prompt_text, return_tensors="pt").to("cuda")

# === Custom generation loop to stop at "yes" ===
def simple_generate_until_yes(model, tokenizer, enc, max_new_tokens=20):
    input_ids = enc["input_ids"]
    attention_mask = enc["attention_mask"]

    generated = []
    for _ in range(max_new_tokens):
        with torch.inference_mode():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits[:, -1, :]
            next_token = torch.argmax(logits, dim=-1, keepdim=True)

        input_ids = torch.cat([input_ids, next_token], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones_like(next_token)], dim=-1)

        decoded = tokenizer.decode(input_ids[0], skip_special_tokens=True)

        # 🚨 Stop immediately if "yes" appears
        if "yes" in decoded.lower():
            return "yes"

    return decoded.strip()

# Run it
output_text = simple_generate_until_yes(model, tokenizer, enc, max_new_tokens=20)
print(output_text)







yes


In [ ]:
import re, torch

def simple_generate(model, tokenizer, enc, max_new_tokens=120):
    input_ids = enc["input_ids"]
    attn = enc["attention_mask"]

    for _ in range(max_new_tokens):
        with torch.inference_mode():
            out = model(input_ids=input_ids, attention_mask=attn)
            logits = out.logits[:, -1, :]
            next_token = torch.argmax(logits, dim=-1, keepdim=True)

        input_ids = torch.cat([input_ids, next_token], dim=-1)
        attn = torch.cat([attn, torch.ones_like(next_token)], dim=-1)

        if next_token.item() == tokenizer.eos_token_id:
            break

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

# === Run test predictions ===
for i, sample in enumerate(test_json[:10], start=1):  # adjust [:10] as needed
    smiles = sample["smiles"]
    true_eff = sample["transfection_efficiency"]
    true_score = sample["score"]

    enc = build_test_inputs(smiles)
    full_text = simple_generate(model, tokenizer, enc, max_new_tokens=120).strip()

    # --- keep only *new text after last SMILES: line* ---
    if f"SMILES: {smiles}" in full_text:
        gen_text = full_text.split(f"SMILES: {smiles}")[-1].strip()
    else:
        gen_text = full_text

    lines = [l.strip() for l in gen_text.splitlines() if l.strip()]
    pred_eff, pred_score, rationale = None, None, ""

    if len(lines) >= 1:
        m = re.search(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", lines[0])
        if m:
            try: pred_eff = float(m.group(0))
            except: pass
    if len(lines) >= 2:
        m = re.search(r"\d+", lines[1])
        if m:
            try: pred_score = int(m.group(0))
            except: pass
    if len(lines) > 2:
        rationale = "\n".join(lines[2:])

    # === Print result ===
    print(f"\n=== Test Sample {i} ===")
    print("SMILES:", smiles)
    print(f"Predicted efficiency: {pred_eff}")
    print(f"Actual efficiency:    {true_eff}")
    print(f"Predicted score:      {pred_score}")
    print(f"Actual score:         {true_score}")
    print("Rationale:", rationale)



=== Test Sample 1 ===
SMILES: OC1=CC=CC=C1CN(CCCN(C)C)CC2=C(O)C=CC=C2
Predicted efficiency: 0.0
Actual efficiency:    2.900367
Predicted score:      None
Actual score:         1.0
Rationale: 

=== Test Sample 2 ===
SMILES: O=C1C(C(OCCCC)=O)CN(CC)CC1C(OCCCC)=O
Predicted efficiency: 0.0
Actual efficiency:    1.20412
Predicted score:      None
Actual score:         1.0
Rationale: 

=== Test Sample 3 ===
SMILES: OC1=CC=C(CCCCCCCCC)C=C1CN(CCCN(CC2=CC(CCCCCCCCC)=CC=C2O)CC3=CC(CCCCCCCCC)=CC=C3O)CC4=C(O)C=CC(CCCCCCCCC)=C4
Predicted efficiency: 0.0
Actual efficiency:    4.722987
Predicted score:      None
Actual score:         2.0
Rationale: 

=== Test Sample 4 ===
SMILES: O=C1C(C(OCCCCCCCC)=O)CN(CN(C)C)CC1C(OCCCCCCCC)=O
Predicted efficiency: 0.0
Actual efficiency:    2.795185
Predicted score:      None
Actual score:         1.0
Rationale: 

=== Test Sample 5 ===
SMILES: OC1=CC=C(CCCCCC)C=C1CN(CCCCCCN2CCN(CCNCCCCCCN(CC3=CC(CCCCCC)=CC=C3O)CC4=CC(CCCCCC)=CC=C4O)CC2)CC5=C(O)C=CC(CCCCCC)=C5
Predic